In [40]:
import builtins
import functools
from typing import Callable, Generic, Iterable, List, ParamSpec, TypeVar

T = TypeVar('T')
U = TypeVar('U')
V = TypeVar('V')
W = TypeVar('W')
P = ParamSpec('P')
P2 = ParamSpec('P2')
E = TypeVar('E')

class Pipe(Generic[P, T]):
    """
    Inspired by the pipe library by Julien Palard <julien@python.org>
    Reimplements it to allow for type checks with mypy.

    This basically allows functions to be decorated to be pipable, where the first argument is
    taken from the left of the pipe.

    Instead of `transform(source, arg)` we can do `source | transform(arg)` while implementing
    transform as taking source as a first argument.
    """

    def __init__(self, func: Callable[P, T]):
        """
        Called when the decorator is applied to a function to turn it into a Pipe.
        Also called when the decorated function is used as a Pipe via __call__

        __call__ supplies `self.func` with the arguments given to the decorated func.
        """
        print('INIT', 'function', func)
        self.func = func

        functools.update_wrapper(self, func)

    def __ror__(self, other):
        """
        Called when the pipe operation is used with this type to its right: `other | this`.
        
        Here we do the unwrapping of the lambdas. The final Pipe lambda has no *args or *kwargs but
        passes them from __call__'s args and kwargs to a wrapper function of itself. When applying
        the other, it will feed the other and the other args/kwargs that were passed to the 
        decorated function via __call__
        """
        print('ROR', 'other', other)
        return self.func()(other)

    def __call__(self, *args: P.args, **kwargs: P.kwargs) -> 'Pipe':
        """
        Called when the decorated function is used. Just register args and kwargs in a new 
        wrapper function.
        """
        print('CALL', 'args', args, 'kwargs', kwargs)
        return Pipe(
            lambda *args2, **kwargs2: 
                self.func(*args, *args2, **kwargs, **kwargs2)
        )

@Pipe
def mapp(callback: Callable[[T], U]):
    def inner(iter: Iterable[T]) -> map:
        return builtins.map(callback, iter)
    return inner

@Pipe
def to_list():
    def inner(iter: Iterable[T]) -> List[T]:
        return list(iter)
    return inner

INIT function <function mapp at 0x7f28ee66b2e0>
INIT function <function to_list at 0x7f28ee566980>


In [39]:
from typing import cast


[1,2,3] | mapp(lambda n: n*3) | to_list()

['A', 'B', 'C'] | mapp(cast(Callable[[str], str], lambda c: c))

CALL args (<function <lambda> at 0x7f28ee6e68e0>,) kwargs {}
INIT function <function Pipe.__call__.<locals>.<lambda> at 0x7f28ee5c9800>
ROR other [1, 2, 3]
CALL args () kwargs {}
INIT function <function Pipe.__call__.<locals>.<lambda> at 0x7f28ee5c9800>
ROR other <map object at 0x7f28ef34dae0>


[3, 6, 9]